In [1]:
# Resources used in our project
# https://en.wikipedia.org/wiki/List_of_United_States_Air_Force_installations -Scraped to get USAF Instalations
# https://github.com/jasonong/List-of-US-States/blob/master/states.csv --CSV to match state names to abbreviations
# statecancerprofiles.cancer.gov --Cancer statistics CSV
#


In [143]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, String, Float
from sqlalchemy.ext.declarative import declarative_base
import pymysql
from config import passw


In [10]:
#URL to scrape
url = 'https://en.wikipedia.org/wiki/List_of_United_States_Air_Force_installations'

In [36]:
#assigning values to new list to scrape, then displaying scraped page as a list
tables = pd.read_html(url)
tables

[                                               0
 0  List of United States Air Force installations
 1    Part of United States Department of Defense
 2                    Map of Air Force Facilities,
                                           0
 0  Map all coordinates using: OpenStreetMap
 1        Download coordinates as: KML · GPX,
                                          0                           1      2  \
 0                                     Name                    Location  State   
 1                     Altus Air Force Base                       Altus     OK   
 2                    Arnold Air Force Base                   Tullahoma     TN   
 3                 Barksdale Air Force Base                Bossier City     LA   
 4                     Beale Air Force Base                  Marysville     CA   
 5                   Buckley Air Force Base                      Aurora     CO   
 6                    Cannon Air Force Base                      Clovis     NM   
 7     

In [243]:
#convert data from list into into a new dataframe, assigning column headers
df = tables[2]
df.columns = ['name', 'location', 'state', 'coordinates', 'major command','unit emblem','unit designation', 'notes']
df.head()

,name,location,state,coordinates,major command,unit emblem,unit designation,notes
0,Name,Location,State,Coordinates,Major Command,Unit Emblem,Unit Designation,Notes/Mission
1,Altus Air Force Base,Altus,OK,34°39′59″N 099°16′05″W﻿ / ﻿34.66639°N 99.26806°W,AETC,NaN,97th Air Mobility Wing,19th Air Force. This wing is tasked to train C...
2,Arnold Air Force Base,Tullahoma,TN,35°23′33″N 086°05′09″W﻿ / ﻿35.39250°N 86.08583°W,AFMC,NaN,Arnold Engineering Development Center,Non-flying base; Primary AFMC research and dev...
3,Barksdale Air Force Base,Bossier City,LA,32°30′07″N 093°39′46″W﻿ / ﻿32.50194°N 93.66278°W,AFGSC,NaN,2d Bomb Wing 307th Bomb Wing (AFRC),The 2d Bomb Wing is the oldest bomb wing in th...
4,Beale Air Force Base,Marysville,CA,39°08′10″N 121°26′11″W﻿ / ﻿39.13611°N 121.43639°W,ACC,NaN,9th Reconnaissance Wing 940th Wing (AFRC),"12th Air Force. U-2R Dragonlady, RQ-4 Global H..."


In [247]:

#remove the unnecessary duplicate header row and then removing unnecessary data columns 
afb_df=df[1:]

afb_final=afb_df[['name','location', 'state','unit designation']].copy()
afb_final.head()

,name,location,state,unit designation
1,Altus Air Force Base,Altus,OK,97th Air Mobility Wing
2,Arnold Air Force Base,Tullahoma,TN,Arnold Engineering Development Center
3,Barksdale Air Force Base,Bossier City,LA,2d Bomb Wing 307th Bomb Wing (AFRC)
4,Beale Air Force Base,Marysville,CA,9th Reconnaissance Wing 940th Wing (AFRC)
5,Buckley Air Force Base,Aurora,CO,460th Space Wing 140th Wing (CO ANG)


In [234]:
#resetting the index to the name column
df.reset_index(inplace=True)
df.head()

,name,location,state,coordinates,major command,unit emblem,unit designation,notes
0,Arnold Air Force Base,Tullahoma,TN,35°23′33″N 086°05′09″W﻿ / ﻿35.39250°N 86.08583°W,AFMC,NaN,Arnold Engineering Development Center,Non-flying base; Primary AFMC research and dev...
1,Barksdale Air Force Base,Bossier City,LA,32°30′07″N 093°39′46″W﻿ / ﻿32.50194°N 93.66278°W,AFGSC,NaN,2d Bomb Wing 307th Bomb Wing (AFRC),The 2d Bomb Wing is the oldest bomb wing in th...
2,Beale Air Force Base,Marysville,CA,39°08′10″N 121°26′11″W﻿ / ﻿39.13611°N 121.43639°W,ACC,NaN,9th Reconnaissance Wing 940th Wing (AFRC),"12th Air Force. U-2R Dragonlady, RQ-4 Global H..."
3,Buckley Air Force Base,Aurora,CO,39°42′06″N 104°45′06″W﻿ / ﻿39.70167°N 104.75167°W,AFSPC,NaN,460th Space Wing 140th Wing (CO ANG),"Non-Flying wing; provides missile warning, mis..."
4,Cannon Air Force Base,Clovis,NM,34°22′58″N 103°19′20″W﻿ / ﻿34.38278°N 103.32222°W,AFSOC,NaN,27th Special Operations Wing,Special Operations base; SOW plans and execute...


In [45]:
# Loading CSV for state abbreviations
# Load in file
states = "Resources/states.csv"
states_pd = pd.read_csv(states)
states_pd.head()


,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [107]:
# Load in cancer data from CSV

cd = "Resources/cd.csv"
cd_df = pd.read_csv(cd)
cd_df.head()


,State,FIPS,Met Healthy People Objective of ***?,"Age-Adjusted Incidence Rate(Ü) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend (á) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1
0,"US (SEER+NPCR)(1,10)",0.0,***,14.5,14.4,14.5,47777,stable,0.6,-0.9,2.1
1,"Puerto Rico(6,10)",72001.0,***,27.4,26.7,28.2,1040,rising,10.5,4.1,17.4
2,"Massachusetts(6,10)",25000.0,***,20.4,19.9,20.9,1458,stable,0.1,-6.5,7.3
3,"Pennsylvania(6,10)",42000.0,***,20.3,20,20.7,2783,falling,-2.3,-4.3,-0.2
4,"New York(6,10)",36000.0,***,19.9,19.6,20.2,4153,stable,2.3,-0.1,4.8


In [176]:
#cleaning up the imported cancer data so we can match by state- splitting on the delimiter '('
new_df = pd.DataFrame(columns=['statename','rem'])
new_df[['statename','rem']] = cd_df['State'].str.split('(', n=1, expand=True)

new_df=new_df[:-27]
cd_df=cd_df[:-27]


In [188]:
combined_df=pd.concat([new_df,cd_df],axis=1)
combined_df.set_index('statename')

final_df= pd.merge(combined_df, states_pd, left_on='statename', right_on='State')

final_df.head()

,statename,rem,State_x,FIPS,Met Healthy People Objective of ***?,"Age-Adjusted Incidence Rate(Ü) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend (á) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,State_y,Abbreviation
0,Massachusetts,"6,10)","Massachusetts(6,10)",25000.0,***,20.4,19.9,20.9,1458,stable,0.1,-6.5,7.3,Massachusetts,MA
1,Pennsylvania,"6,10)","Pennsylvania(6,10)",42000.0,***,20.3,20,20.7,2783,falling,-2.3,-4.3,-0.2,Pennsylvania,PA
2,New York,"6,10)","New York(6,10)",36000.0,***,19.9,19.6,20.2,4153,stable,2.3,-0.1,4.8,New York,NY
3,Connecticut,"3,8)","Connecticut(3,8)",9000.0,***,19.6,19,20.3,753,stable,1.6,0,3.2,Connecticut,CT
4,Utah,"3,8)","Utah(3,8)",49000.0,***,19.3,18.5,20,503,stable,-0.2,-5.9,6,Utah,UT


In [228]:
## Creating a final dataframe with only the columns necessary for our data

clean_df=final_df[['statename','Age-Adjusted Incidence Rate(Ü) - cases per 100,000', 
                      'Lower 95% Confidence Interval', 'Upper 95% Confidence Interval', 'Average Annual Count',
                     'Recent Trend','Recent 5-Year Trend (á) in Incidence Rates','Lower 95% Confidence Interval.1',
                     'Upper 95% Confidence Interval.1','Abbreviation']].copy()




In [229]:
clean_df.head()

,statename,"Age-Adjusted Incidence Rate(Ü) - cases per 100,000",Lower 95% Confidence Interval,Upper 95% Confidence Interval,Average Annual Count,Recent Trend,Recent 5-Year Trend (á) in Incidence Rates,Lower 95% Confidence Interval.1,Upper 95% Confidence Interval.1,Abbreviation
0,Massachusetts,20.4,19.9,20.9,1458,stable,0.1,-6.5,7.3,MA
1,Pennsylvania,20.3,20,20.7,2783,falling,-2.3,-4.3,-0.2,PA
2,New York,19.9,19.6,20.2,4153,stable,2.3,-0.1,4.8,NY
3,Connecticut,19.6,19,20.3,753,stable,1.6,0,3.2,CT
4,Utah,19.3,18.5,20,503,stable,-0.2,-5.9,6,UT


In [252]:
## inserting two pandas dataframes into MYSQL

##Importing dependencies

#creating connection to mysql/creating engine for mysql to insert

pymysql.install_as_MySQLdb()
Base = declarative_base()

user = 'root'
host =  '127.0.0.1'
port = 3306
database = 'etl_db'

engine = create_engine(f'mysql://{user}:{passw}@{host}:{port}/{database}')
conn = engine.connect()

# submitting two dataframes to mysql server 

clean_df.to_sql('states', con=conn, if_exists = 'replace', index=False)
afb_final.to_sql('afb', con=conn, if_exists = 'replace', index=False)
